In [18]:
import boto3
from botocore.exceptions import ClientError
import configparser

config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key = config['AWS']['aws_access_key_id']
aws_secret_key = config['AWS']['aws_secret_access_key']


In [19]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1", aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

# Set the model ID, e.g., Titan Text Premier.
model_id = "amazon.titan-text-express-v1"

In [22]:
import pandas as pd
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

landmarks = s3.download_file('capstone-techcatalyst-raw', 'group1/Individual_Landmark_Sites_20240801.csv', 'Individual_Landmark_Sites_20240801.csv')
landmark_df = pd.read_csv('/workspaces/thePANDAsCapstone/Pipeline Stages/Databricks/Individual_Landmark_Sites_20240801.csv')

In [59]:
landmark_list = landmark_df['LPC_NAME'].to_list()

length = len(landmark_list)

landmark_str = ', '.join(landmark_list[0:length//2])


query = f""" {landmark_str}

From this list of landmarks, create a list of 5 categories of landmarks. 


"""



TypeError: sequence item 0: expected a bytes-like object, str found

In [57]:
query

" 105 Franklin Avenue House, Decker Farmhouse, Public School 15 (Daniel D. Tompkins School), 121 Heberton Avenue House, Reverend David Moore House, 752 Delafield Avenue, Mary and David Burgher House, Staten Island Borough Hall, Nathaniel J. and Ann C. Wyeth House, 22 Pendleton Place House, 364 Van Duzer Street House, Staten Island Family Courthouse, John DeGroot House, Saint Andrew's Church, Stephen D. Barnes House, 66 Harvard Avenue House, John King Vanderbilt House, New Dorp Light, Expanded Site, 390 Van Duzer Street House, 411 Westervelt Avenue House, Horton's Row, H. H. Richardson House, August and Augusta Schoverling House, Julia Gardiner Tyler House, Saint John's Church, 29 Cottage Place House, Gustave A. Mayer House, Reformed Church on Staten Island, Sunday School Building, and Cemetery, Cornelius Cruser House, 1 Pendleton Place House, 417 Westervelt Avenue House, Horton's Row, William and Catherine Cass House, The Crimson Beech, Tysen-Neville House, 413 Westervelt Avenue House,

In [58]:
# Start a conversation with the user message.
user_message = query
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId="amazon.titan-text-express-v1",
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
        additionalModelRequestFields={}
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Here are 5 categories of landmarks:

1. Houses
2. Churches
3. Theatres
4. Museums
5. Libraries
